In [1]:
#Parameters setup

In [2]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.python.ops import rnn, rnn_cell
import numpy as np
import data_helpers
from tensorflow.contrib import learn
import os


# Parameters
# ==================================================


# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularizaion lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 40, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 400, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 400, "Save model after this many steps (default: 100)")
# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.flags.DEFINE_boolean("stop_step", 400, "model training times")
tf.flags.DEFINE_string("event", "sydneysiege", "event name (default: sydneysiege)")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")

event=FLAGS.event
fo=open(event+"_rst.txt","w")


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=64
CHECKPOINT_EVERY=400
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=400
EVENT=sydneysiege
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NUM_EPOCHS=40
NUM_FILTERS=128
STOP_STEP=400



In [3]:
# Data Preparatopn

In [4]:
# Load data
print("Loading data...")
x_text, y_label = data_helpers.load_data_and_labels()

#class_list=["ottawashooting","charliehebdo","ferguson","sydneysiege"]

my_list=[]

for x in xrange(len(x_text)):
    i=x_text[x].find('*')
    sub=x_text[x][:i]
    x_text[x]=x_text[x][i+1:]
    if(sub==event):
        i1=x_text[x].find('*')
        sub1=x_text[x][:i1]#threadID      
        #x_text[x]=x_text[x][i1:]      
        my_list.append(sub1)

my_list = list(set(my_list)) #threadID list  
test_num=[]
for rum in my_list:            
    temp=[]    
    for x in xrange(len(x_text)):
        i1=x_text[x].find('*')
        if (i1!=-1):
            if(rum==x_text[x][:i1]):
                temp.append(x)
                #x_text[x]=x_text[x][i1:]
    test_num.append(temp)

#data sub string
for x in xrange(len(x_text)):
    x_text[x]=x_text[x][19:]

Loading data...


In [5]:
#build word2vec

In [6]:
outfile="vocab"
max_document_length = max([len(x.split(" ")) for x in x_text]) # Get vocabulary length
if os.path.isfile(outfile+".npy"):
    x_=np.load(outfile+".npy")
else:
   
    #Return word2vec
    sub_vec=data_helpers.word_vec_setup()
    x_vec=[]
    for k in x_text:
        k_sp=k.split(" ")#each word in instance
        sent_vec=[]
        i=-1
        for _ in xrange(len(k_sp)):
            i+=1    
            sent_vec=np.append(sent_vec,data_helpers.word_vec_lookup(k_sp[i],sub_vec))
        for _ in xrange(i+1,max_document_length):
            sent_vec=np.append(sent_vec,np.array([0]*25))        
        x_vec=np.append(x_vec,sent_vec)
    x_vec.shape=(-1,max_document_length,25)
    np.save(outfile, x_vec)#save vec to file
    x_=x_vec

In [7]:
# Parameters
learning_rate = 0.001
training_iters = 10
batch_size = FLAGS.batch_size
display_step = 100

# Network Parameters
n_input = 25 # MNIST data input (img shape: 28*28)
n_steps = max_document_length # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 4 # MNIST total classes (0-9 digits)

model_n=0

In [8]:
#training 

In [9]:
for n in test_num:

    model_n+=1
    x_train,y_train=x_,y_label
    x_dev,y_dev=[],[]
    inplace=0

    for num in n:
        x_dev.append(x_[num])
        y_dev.append(y_label[num])
        x_train = np.delete(x_train, num-inplace,0)
        y_train = np.delete(y_train, num-inplace,0) 
        inplace+=1
        #print(x_text[num])
        #print(x_[num])

    #print("Vocabulary Size: {:d}".format(len(vocab_processor.vocabulary_)))
    print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))


        # tf Graph input
    x = tf.placeholder("float", [None, n_steps, n_input])
    y = tf.placeholder("float", [None, n_classes])

    # Define weights
    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    def print_full(arr):
        print ("len(arr):"+str(len(arr)))
        print ("len(arr[0]):"+str(len(arr[0])))
        aa=""
        for i in xrange(len(arr)):
            for j in xrange(len(arr[0])):
                aa+=str(arr[i][j])+" "
            print (aa)
            print ("------------------------")

    def RNN(x, weights, biases,model_n):
        # Prepare data shape to match `rnn` function requirements
        # Current data input shape: (batch_size, n_steps, n_input)
        # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)

        # Permuting batch_size and n_steps
        x = tf.transpose(x, [1, 0, 2])
        # Reshaping to (n_steps*batch_size, n_input)
        x = tf.reshape(x, [-1, n_input])
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
        x = tf.split(0, n_steps, x)
	with tf.variable_scope("myrnn", reuse = None) as scope:
            # Define a lstm cell with tensorflow
	    if (model_n>1):
	        scope.reuse_variables()#not to share variable
            lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0,state_is_tuple=True)
            # Get lstm cell output
            outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

            # Linear activation, using rnn inner loop last output
        return tf.matmul(outputs[-1], weights['out']) + biases['out']

    pred = RNN(x, weights, biases,model_n)


    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initializing the variables
    init = tf.initialize_all_variables()

    # Launch the graph
    with tf.Session() as sess:
	fo=open(event+"_rst.txt","a")
        sess.run(init)
        step = 1
        # Keep training until reach max iterations
        #while step * batch_size < training_iters:
        batches = data_helpers.batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        for batch in batches:
            x_batch, y_batch = zip(*batch)  

	    x_batch=np.array(x_batch)
	    y_batch=np.array(y_batch)
           #print_full(batch_x[0])
              # Run optimization op (backprop)
            sess.run(optimizer, feed_dict={x: x_batch, y: y_batch})
            if step % display_step == 0:
                # Calculate batch accuracy
                acc = sess.run(accuracy, feed_dict={x: x_batch, y: y_batch})
                # Calculate batch loss
                loss = sess.run(cost, feed_dict={x: x_batch, y: y_batch})
                print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
            step += 1
        print("Optimization Finished!")

        # Calculate accuracy 

        pred,accuracy=sess.run([pred,accuracy],feed_dict={x: x_dev, y: y_dev})
        print(accuracy)
        for i in xrange(len(pred)):
	    if (y_dev[i][0]==1):
                fo.write (str(pred[i])+", 0\n")
	    elif (y_dev[i][1]==1):
                fo.write (str(pred[i])+", 1\n")
	    elif (y_dev[i][2]==1):
                fo.write (str(pred[i])+", 2\n")
	    elif (y_dev[i][3]==1):
                fo.write (str(pred[i])+", 3\n")
        fo.write("\n")

Train/Dev split: 4116/33
Iter 6400, Minibatch Loss= 1.231309, Training Accuracy= 0.35938
Iter 12800, Minibatch Loss= 1.109511, Training Accuracy= 0.48438
Iter 19200, Minibatch Loss= 0.970205, Training Accuracy= 0.50000
Iter 25600, Minibatch Loss= 0.964686, Training Accuracy= 0.51562
Iter 32000, Minibatch Loss= 1.015546, Training Accuracy= 0.59375
Iter 38400, Minibatch Loss= 0.793026, Training Accuracy= 0.67188
Iter 44800, Minibatch Loss= 0.986932, Training Accuracy= 0.54688
Iter 51200, Minibatch Loss= 0.840215, Training Accuracy= 0.60938
Iter 57600, Minibatch Loss= 0.787711, Training Accuracy= 0.73438
Iter 64000, Minibatch Loss= 0.697850, Training Accuracy= 0.68750
Iter 70400, Minibatch Loss= 0.542169, Training Accuracy= 0.84375
Iter 76800, Minibatch Loss= 0.533205, Training Accuracy= 0.76562
Iter 83200, Minibatch Loss= 0.420205, Training Accuracy= 0.79688
Iter 89600, Minibatch Loss= 0.446040, Training Accuracy= 0.82812
Iter 96000, Minibatch Loss= 0.477115, Training Accuracy= 0.79688
I

KeyboardInterrupt: 